*Author: C Mitchell*

**This notebook deals with the TNC Basin, Pemaquid and Bigelow data.**

The `sample_list_v1.csv` was initially created from the notes in the hand-written lab book (and contains the PSR TNC Basin and Pemaquid data, and the SVIS Pemaquid2 data, but not the leafclip data).

In this notebook, we add the time columns and the number of measurements for each sample into the `sample_list_v1.csv` file.

**The output of this notebook is a new file, `sample_list_v2.csv`**

# Time details

For some of the lab experiments, we were investigation the effect of desiccation i.e. time to dry. Each of the raw PSR .sed files have time in the metadata/header information. We're going to use these time stamps to include the `sample_list_v1.csv` file.

For each sample, we have 40 measurements, so we have 40 time stamps. We are going to use the time stamp from the *first measurement* of each sample.

Note for the SVIS Bigelow data we did no drying, so I'm not including time stamps on those measurements

# Initializing workflow

Importing modules and setting up the data directories

In [1]:
import pandas as pd
import os
import re
import datetime as dt

In [2]:
datadirs = {'Basin' : '/mnt/storage/labs/mitchell/projects/meifsci-seaweed-drones/analysis/data/20210622_TNCBasin/raw/', 
            'Pemaquid' : '/mnt/storage/labs/mitchell/projects/meifsci-seaweed-drones/analysis//data/20210721_Pemaquid/raw/labday/',
           'Bigelow' : '/mnt/storage/labs/mitchell/projects/meifsci-seaweed-drones/analysis/data/20211209_Bigelow/raw/'}

## Defining functions

In [3]:
def groupsed(datadir):
    # finds all the .sed files from a given data dir and
    # groups them by filestem
    #
    # inputs: full path to directory with the .sed files
    # outputs: a dictionary with the filestem as the key
    #          and the list of corresponding individual 
    #          files as the value
    
    allfiles = os.listdir(datadir)
    datafiles = [f for f in allfiles if re.search('.sed',f)]

    ind = [re.search('_[0-9]{5}\.sed',f).start() for f in datafiles]
    fstem = sorted(set([f[:ii] for f,ii in zip(datafiles,ind)]))
    filegroups = {}
    for fs in fstem:
        filegroups[fs] = [datadir+f for f in datafiles if re.match(fs+'_[0-9]',f)]
    
    return filegroups

In [4]:
def sedtimestamp(filename):
    # finds the time information in the header info and
    # pulls out the hours, minutes and seconds of the measurement
    # inputs: full path to the .sed file
    # outputs: hour, minutes and seconds as integers
    
    with open(filename) as fp:
        Lines = fp.readlines()
        for line in Lines:
            if re.match('Time: ', line):
                parsed = re.split(':',line)
                hr = int(re.split(',',parsed[3])[1])
                mins = int(parsed[4])
                secs = int(float(parsed[5]))
    
    return hr, mins, secs

# Processing files

Now we can loop through our different samples and pull out the time stamps and count how many measurements there were for each sample.

In [5]:
sample_info = {}
for sitekey,datadir in datadirs.items():
    if not re.match('Bigelow', sitekey):
        filegroups = groupsed(datadir)

        for fg,flist in filegroups.items():
            hrs, mins, secs  = sedtimestamp(flist[0])
            sample_info[sitekey+'_'+fg]  = [hrs, mins, secs, len(flist)]

And finally, we can put this together into a data frame:

In [6]:
new_sample_info = pd.DataFrame(sample_info,index =['hours','minutes','seconds','n']).T.rename_axis('idkey')

In [7]:
new_sample_info

,hours,minutes,seconds,n
idkey,,,,
Basin_Asco1,10,53,31,40
Basin_Asco1b,13,51,31,40
Basin_Asco1c,15,19,45,40
Basin_Asco2,11,0,12,40
Basin_Asco2b,13,58,50,40
...,...,...,...,...
Pemaquid_Fucus8,15,37,41,40
Pemaquid_Fucus9,16,17,41,8
Pemaquid_WhiteRef4,13,0,13,1


# Combining the new sample information with `sample_list_v1.csv`

We need to add the time stamps and number of measurements to the existing sample information in the `sample_list_v1.csv` file.

First, we need to import `sample_list_v1.csv`

In [8]:
sampledf = pd.read_csv('../data/sample_list_v1.csv')

In [9]:
sampledf

,Site,Sample,Wet_weight,Dry_weight,Canopy_Depth
0,Basin,Asco1b,39.0,13.3,1.0
1,Basin,Asco1c,39.0,13.3,1.0
2,Basin,Asco1,39.0,13.3,1.0
3,Basin,Asco2b,83.3,28.7,2.8
4,Basin,Asco2c,83.3,28.7,2.8
...,...,...,...,...,...
78,Bigelow,fucus6,591.2,NaN,9.0
79,Bigelow,fucus7,655.3,NaN,9.0
80,Bigelow,fucus8,698.8,NaN,11.0
81,Bigelow,fucus_salad,93.9,NaN,4.1


To merge `sampledf` with `new_sample_info`, we need a unique, identifying key to merge on. The unique key will be `site_sample`. The `new_sample_info` already has this key as a column, but we need to add that key to `sampledf`

In [10]:
sampledf['idkey'] = sampledf['Site'] + '_' + sampledf['Sample']
sampledf.set_index('idkey')

,Site,Sample,Wet_weight,Dry_weight,Canopy_Depth
idkey,,,,,
Basin_Asco1b,Basin,Asco1b,39.0,13.3,1.0
Basin_Asco1c,Basin,Asco1c,39.0,13.3,1.0
Basin_Asco1,Basin,Asco1,39.0,13.3,1.0
Basin_Asco2b,Basin,Asco2b,83.3,28.7,2.8
Basin_Asco2c,Basin,Asco2c,83.3,28.7,2.8
...,...,...,...,...,...
Bigelow_fucus6,Bigelow,fucus6,591.2,NaN,9.0
Bigelow_fucus7,Bigelow,fucus7,655.3,NaN,9.0
Bigelow_fucus8,Bigelow,fucus8,698.8,NaN,11.0


And finally, merge these two dataframes together:

In [11]:
final_sample_list = pd.merge(sampledf,new_sample_info,on='idkey',how='outer').set_index('idkey')

In [12]:
final_sample_list

,Site,Sample,Wet_weight,Dry_weight,Canopy_Depth,hours,minutes,seconds,n
idkey,,,,,,,,,
Basin_Asco1b,Basin,Asco1b,39.0,13.3,1.0,13.0,51.0,31.0,40.0
Basin_Asco1c,Basin,Asco1c,39.0,13.3,1.0,15.0,19.0,45.0,40.0
Basin_Asco1,Basin,Asco1,39.0,13.3,1.0,10.0,53.0,31.0,40.0
Basin_Asco2b,Basin,Asco2b,83.3,28.7,2.8,13.0,58.0,50.0,40.0
Basin_Asco2c,Basin,Asco2c,83.3,28.7,2.8,15.0,27.0,48.0,40.0
...,...,...,...,...,...,...,...,...,...
Basin_Background,NaN,NaN,NaN,NaN,NaN,10.0,34.0,49.0,4.0
Basin_Fucusxx,NaN,NaN,NaN,NaN,NaN,11.0,47.0,40.0,1.0
Pemaquid_WhiteRef4,NaN,NaN,NaN,NaN,NaN,13.0,0.0,13.0,1.0


In [13]:
final_sample_list.drop(['Basin_Fucusxx','Basin_Background','Pemaquid_WhiteRef4','Pemaquid_Whiteref','Pemaquid_Whiteref5'],inplace = True)

In [14]:
final_sample_list

,Site,Sample,Wet_weight,Dry_weight,Canopy_Depth,hours,minutes,seconds,n
idkey,,,,,,,,,
Basin_Asco1b,Basin,Asco1b,39.0,13.3,1.0,13.0,51.0,31.0,40.0
Basin_Asco1c,Basin,Asco1c,39.0,13.3,1.0,15.0,19.0,45.0,40.0
Basin_Asco1,Basin,Asco1,39.0,13.3,1.0,10.0,53.0,31.0,40.0
Basin_Asco2b,Basin,Asco2b,83.3,28.7,2.8,13.0,58.0,50.0,40.0
Basin_Asco2c,Basin,Asco2c,83.3,28.7,2.8,15.0,27.0,48.0,40.0
...,...,...,...,...,...,...,...,...,...
Bigelow_fucus6,Bigelow,fucus6,591.2,NaN,9.0,NaN,NaN,NaN,NaN
Bigelow_fucus7,Bigelow,fucus7,655.3,NaN,9.0,NaN,NaN,NaN,NaN
Bigelow_fucus8,Bigelow,fucus8,698.8,NaN,11.0,NaN,NaN,NaN,NaN


In [15]:
final_sample_list.to_csv('../data/sample_list_v2.csv')